In [1]:
import numpy as np
import matplotlib.pyplot as plt
from findiff import FinDiff, coefficients
from scipy.linalg import solve, det, expm
from math import isclose
import pandas as pd
from scipy.signal import savgol_filter

In [2]:
############ ODE MODEL #################

### FOR FINITE DIFFERENCE ###

def floor_even(num):
    return np.floor(num/2)*2

def floor_odd(num):
    if num%2==1:
        return num
    else:
        return num-1

def highest_possible_N(time_series):
    number = 2/3 * len(time_series) -2/3

    # Due to floating point errors causing np.floor to return the wrong number,
    # (for instance number=65.9999 when it is actually =66), we use isclose 
    if isclose(number,np.ceil(number), abs_tol=1e-8):
        return int(round(number))
    else:
        return int(np.floor(number))

def highest_possible_accuracies(time_series, N):
    acc_list = []
    for deriv in range(1,N+1):
        # Different conditions for odd and even derivatives:
        if deriv%2==1:
            number = 2/3*(len(time_series))+1-deriv
            # Handle floating point error:
            if isclose(number,np.ceil(number), abs_tol=1e-8):
                actual_number=round(number)
                acc_list.append(int(floor_even(actual_number)))
            else:
                acc_list.append(int(floor_even(number)))
        else:
            number = 2/3*(len(time_series))+4/3-deriv
            # Handle floating point error:
            if isclose(number,np.ceil(number), abs_tol=1e-8):
                actual_number=round(number)
                acc_list.append(int(floor_even(actual_number)))
            else:
                acc_list.append(int(floor_even(number)))
                
    return acc_list


def points_to_accuracy(time_series, p_to_use, N):
    print("p_to_use: ", p_to_use)
    # n+m points used in a findiff approx (order + accuracy)
    # Accuracy has to be an even number.
    
    accuracy_list = []
    
    if p_to_use == 'few':
        accuracy_list = [2]*N
        
    if p_to_use == 'all':
        accuracy_list = highest_possible_accuracies(time_series, N)
        
    if '%' in p_to_use:
        percent = float(p_to_use.split("%")[0])*0.01
        ts_len = len(time_series)
        num_points = int(np.round(ts_len*percent))
        
        for n in range(1,N+1):
            if n>num_points-2:
                m = 2
                accuracy_list.append(m)
            elif n%2==1:
                m = int(floor_even(num_points-n))
                while ((n + m - 1)/2 - 1 + n + m) > ts_len:
                    m -= 2
                accuracy_list.append(m)
            elif n%2==0:     
                m = int(floor_even(num_points-n))
                while ((n + m - 1)/2 - 1 - 1 + n + m) > ts_len:
                    m -= 2
                accuracy_list.append(m)    
    return accuracy_list

def derivative_approximations(h, time_series, N, accuracy_list):
    fin_diff = [time_series] # the 0-th derivative is the original time series
    #np.set_printoptions(precision=2)
    for k in range(1,N+1):
        derivative = FinDiff(0, h, k, acc=accuracy_list[k-1])
        fin_diff.append(derivative(time_series))
        
        mat_repr_deriv_approx = derivative.matrix(time_series.shape)
        #print(f"FINITE DIFF. MATRIX REPRESENTATION. deriv={k}, acc={accuracy_list[k-1]}")
        #print(mat_repr_deriv_approx.toarray())
        #print("")
        
        
    return np.asarray(fin_diff)

### FOR DETERMINING COEFFICIENTS ANALYTICALLY ###

def coeff_analytical_matrix_vector(fin_diff, coeff_to_lock):
    N = np.shape(fin_diff)[0]-1 # Highest order derivative
        
    # We will return A and b, as we later wish to solve the matrix equation Ax=b, where the solution x 
    # will contain all the coefficients except the locked one, i.e. c_k: [a, c_0, ..., c_k-1, c_k+1, ..., c_N]:
    A = np.zeros((N+1,N+1)) # LHS MATRIX
    b = np.zeros((N+1, 1))  # RHS vector
        
        
    # Two cases. If coeff_to_lock is 'a':
    if coeff_to_lock == 'a':
        temp_matrix = np.copy(fin_diff)
        for m in range(N+1):
            b[m,0] = -1*np.sum(temp_matrix[m])
            A[m,m] = np.sum(temp_matrix[m]*temp_matrix[m]) 
            for l in range(m+1, N+1):
                A_ml = np.sum(temp_matrix[l]*temp_matrix[m])
                A[m,l] = A_ml
                A[l,m] = A_ml 
        
        # In this case, the final solution x to Ax=b, will look like x=[c_0, c_1, ..., c_k, ..., c_N]
        
    # coeff_to_lock is c_k:
    else:
        coeff_to_lock_index = int(coeff_to_lock.split("_")[1]) # e.g. c_0 corresponds to row 0 in fin_diff
        
        # Code is nicer if we replace row corresponding to locked 
        # coefficient with ones
        dyk_i = np.copy(fin_diff[coeff_to_lock_index])
        temp_matrix = np.delete(fin_diff, coeff_to_lock_index, axis=0)
        temp_matrix = np.insert(temp_matrix, 0, np.ones_like(dyk_i), axis=0)

        for m in range(N+1):
            b[m,0] = -1*np.sum(dyk_i*temp_matrix[m])
            A[m,m] = np.sum(temp_matrix[m]*temp_matrix[m]) 
            for l in range(m+1, N+1):
                A_ml = np.sum(temp_matrix[l]*temp_matrix[m])
                A[m,l] = A_ml
                A[l,m] = A_ml 
                
        # In this case, the final solution x to Ax=b will look like x=[a, c_0, ..., c_k-1, c_k+1, ..., c_N]
    
    return A, b

'''
# Prevent singular matrix
def check_if_deriv_is_zero(fin_diff, tolerance=1e-6):
    # We return the index of the first derivative which is practically zero (we know all the following ones will also be zero).
    sum_of_rows = np.sum(abs(fin_diff), axis=1)
    print("Sum of each row in fin_diff: \n", sum_of_rows)
    for row_number, row_sum in enumerate(sum_of_rows):
        if abs(row_sum) <= tolerance:
            return row_number
    return 'All derivatives nonzero'

def prev_sing_matrix(fin_diff, coefficient_to_lock):
    ### PREVENT SINGULAR MATRIX: This can happen if the time series is sampled from a polynomial with degree < N.
    # If any of the derivative approximations is equal to zero, this will give us a singular matrix below.
    # If the derivs after a certain order are zero, we set the coefficients for these derivs to zero.
    # Then we only work with fin_diff up to and including the final nonzero deriv approx.   
    print("PREVENT SINGULAR MATRIX")
    print("Check if any of the derivatives are 0: \n")
    first_index_deriv_zero = check_if_deriv_is_zero(fin_diff, tolerance=1e-6) # Need a tolerance due to floating point error.
    print("")
    if first_index_deriv_zero!='All derivatives nonzero':
        if first_index_deriv_zero == 0:
            print("The time series is equal to zero.")
            return
        
        if coefficient_to_lock != 'a' and int(coefficient_to_lock.split("_")[1])>=first_index_deriv_zero:
            print("The selected coefficient to lock corresponds to a derivative that is equal to zero.")
            print(f"Select a coefficient c_k to lock with k <= {first_index_deriv_zero-1}, as all derivatives of order")
            print("higher than this are equal to zero.")
            return
        
        fin_diff = fin_diff[:first_index_deriv_zero]
        
        print(f"The highest order derivative to include was given as: {N}.")
        print(f"However, the derivatives of order >= {first_index_deriv_zero} are equal to zero.")
        print("The returned list of coefficients will therefore only include the coefficients up to and including:") 
        print(f"c_{first_index_deriv_zero-1} \n")
    else:
        print("findiff does not contain rows that are equal to zero.")
    return fin_diff

# Prevent linearly dependent matrix
def determine_lin_dep_columns(A, tolerance=1e-6):
    low = 0
    high = A.shape[1]-1
    mid = 0
    
    # We do a binary search until we get the index of the final column to include, whcih corresponds with the "high" variable
    if abs(det(A))<tolerance:
        while low <= high:
            mid = (high+low)//2
            
            if abs(det(A[:mid, :mid]))<tolerance:
                high = mid-1
            else:
                low = mid+1
                
        return high
    else:
        return ("Matrix is linearly independent")
    
def prev_lin_dep_matrix(A, b):
    ### PREVENT LINEARLY DEPENDENT MATRIX: This can happen if a row in fin_diff is a multiple of 
    # an earlier row in fin_diff. I.e. what happens when we have a time series sampled from exp(t),
    # then every dervative is the same. We solve this by finding the largest submatrix in A s.t.
    # A is linearly independent. We do this by checking if the determinant is smaller or bigger
    # than some tolerance.
    print("CHECK IF MATRIX IS LINEARLY INDEPENDENT")
    final_col_to_include = determine_lin_dep_columns(np.copy(A), tolerance=1e-4)
    if final_col_to_include != "Matrix is linearly independent":
        print("The matrix was not linearly independent.")
        print("We extract the largest submatrix starting in the upper left corner that's linearly independent.")
        print(f"The final column from A we include is the column with index: {final_col_to_include-1}.")
        print(f"This gives the matrix A[:{final_col_to_include},:{final_col_to_include}]:")
        
        A = A[:final_col_to_include, :final_col_to_include]
        b = b[:final_col_to_include]
        
        print("A: \n", A)
        print("b: \n", b)
        
        print(f"Since the matrix {final_col_to_include} rows/columns corresponds to having originally")
        print(f"chosen N={final_col_to_include-1} instead, the returned list of coefficients")
        print(f"will therefore only include coefficients up to and including: c_{final_col_to_include-1}")
    else:
        print(final_col_to_include)
    return A, b
'''


def insert_locked_coeff(solved_coeff, coefficient_to_lock):
    if coefficient_to_lock == 'a':
        solved_all_coeff = np.insert(solved_coeff, 0, 1, axis=0) 
        # Here, solved_all_coeff looks like [1, c_0, ..., c_k, ..., c_N]
    else: 
        coeff_to_lock_index = int(coefficient_to_lock.split("_")[1])+1 # +1 since 'a' is at index 0.
        solved_all_coeff = np.insert(solved_coeff, coeff_to_lock_index, 1, axis=0)
        # Here, if c_k is the locked coeff, solved_all_coeff looks like [a, c_0, ..., c_k-1, 1, c_k+1, ..., c_N].
    return solved_all_coeff
        
# Find the coefficients solving the minimization problem
def solved_all_coeffs_analytical(time_points, time_series, N, acc_p_to_use, coefficient_to_lock='c_0', accuracy_list=None):
    time_points = np.array(time_points)
    time_series = np.array(time_series)
    
    ### Check if chosen N is possible:
    highest_pos_N = highest_possible_N(time_series)
    if N<=highest_pos_N:
        print(f"The chosen highest order derivative is N={N}, which is less than or equal")
        print(f"to the highest possible derivative, N_highest = {highest_pos_N}")
    else:
        print(f"The chosen highest order derivative is N={N}, which is greater than")
        print(f"the highest possible derivative, N_highest = {highest_pos_N}")
        return
    
    # Check if coefficient to lock is valid
    if coefficient_to_lock != 'a' and coefficient_to_lock[:2]!='c_':
        print("coefficient_to_lock must be either 'a' or 'c_k', for 0 <= k <= N")
        return
    
    if coefficient_to_lock[:2]=='c_' and (int(coefficient_to_lock.split("_")[1])>N or int(coefficient_to_lock.split("_")[1])<0):
        print("c_k: k must be between 0 and N")
        return
    
    ### Create a list with the highest possible accuracy for each derivative approx:
    if accuracy_list == None:
        accuracy_list = points_to_accuracy(time_series, acc_p_to_use, N)
    
    print("\n List containing order of accuracy for the derivative approximations.")
    print("Index i corresponds to the accuracy of the derivative of order i+1: \n", accuracy_list, "\n")
    
    ### Create an array with all derivative approximations. 
    ### The row corresponds to the derivative order, and column corresponds to the time point.
    ### Ex: value at (2, 1), is the approximation of the second derivative of y at time t_1, that is y_1''.
    fin_diff = derivative_approximations(time_points[1]-time_points[0], time_series, N, accuracy_list)  
    print("\n fin_diff: \n", fin_diff, "\n")
    ###  fin_diff - numpy array with  shape (N+1, n+1): 
    ###    [ [y_0     ,   y_1      ,   ...   ,    y_n     ],
    ###      [y_0'    ,   y_1'     ,   ...   ,    y_n'    ],
    ###      [y_0''   ,   y_1''    ,   ...   ,    y_n''   ],
    ###                     ...,
    ###      [y_0^(N) ,   y_1^(N)  ,   ...   ,    y_n^(N) ]  ] 
    
    
    #fin_diff = prev_sing_matrix(fin_diff, coefficient_to_lock)
    
    print("CONSTRUCT MATRIX FOR MATRIX EQUATION")
    A, b = coeff_analytical_matrix_vector(fin_diff, coefficient_to_lock) 
    print("A: \n", A)
    print("b: \n", b)
    print("")    
    #A, b = prev_lin_dep_matrix(A,b)
    print("det of A: ", det(A))
    
    solved_coeff = solve(A, b, assume_a='sym') # Column vector. We use the convention ["a","c_0", "c_1", "c_2", ..., "c_N"]^T
    
    # The solution to the matrix equation Ax = b does not contain the locked coefficient, so we have to add it back.
    # Since this is the coefficient that's "divided out", it has value 1:
    solved_all_coeff = insert_locked_coeff(solved_coeff, coefficient_to_lock)
    
    return solved_all_coeff, fin_diff

### FOR CALCULATING THE EXPLICIT SOLUTION

# Find the matrix and vector for converting ODE to system of first order equations
def coeff_list_to_system_matrix(all_coeffs, tolerance):
    # all_coeffs = np.array([[a], [1], [c_1], [c_2], ..., [c_N]]) where c_0 is the locked coeff.
    
    # As we will divide by c_N, we must ensure that the last coeff in the list is nonzero.
    # This is OK, becaue if the array ends with c_{k+1}, ..., c_{N} that are (approx) equal to zero
    # we in practice have an ODE of order k, so we can just drop these last terms. 
    while abs(all_coeffs[-1,0])<tolerance:
        all_coeffs = all_coeffs[:-1]
    
    print("remove highest order coeff(s) if below threshold: \n", all_coeffs)
    
    # The order of the ODE determines the shape of the matrix.
    ode_order = all_coeffs.shape[0]-2 # -2 because we have the constant term and the y^(0) coeff.
    print("If any coefficients were removed, the effective ODE order will change. The ODE order is now: ", ode_order)
    if ode_order == 0:
        C = None
        A = None
        return C, A
        
    # Create matrix C
    C = -1*np.eye(ode_order, k=1)
    bottom_row = all_coeffs[1:-1,0].T/all_coeffs[-1,0]
    print("bottom_row: ", bottom_row)
    C[-1, :] = bottom_row 
    print("C: \n", C)
    # Create vector A
    A = np.zeros((ode_order,1))
    A[-1,0]=all_coeffs[0,0]
    print("A: \n", A)
    return C, A

def initial_cond(fin_diff, percentage_of_interval, where_to_start, N):
    num_points = floor_odd(int(np.floor(fin_diff.shape[1]*0.01*float(percentage_of_interval.split('%')[0]))))
    print("num_points: ", num_points)
    t0_index = 0
    
    if where_to_start == 'start':
        t0_index = num_points//2 + 1
    
    if where_to_start == 'middle':
        t0_index = fin_diff.shape[1]//2
        
    if where_to_start == 'end':
        t0_index = fin_diff.shape[1] - (num_points//2 + 1)
    
    print("t0_index: ", t0_index)
    
    y_initial = [np.mean(fin_diff[0,t0_index-num_points//2:t0_index+num_points//2+1])]
    for n in range(1,N):
        if n%2==1:
            new_num_points = num_points-(2+n)
        else:
            new_num_points = num_points-(3+n)
        
        y_initial.append(np.mean(fin_diff[n,t0_index-new_num_points//2:t0_index+new_num_points//2+1]))
    
    return np.reshape(y_initial,(-1,1)), t0_index

# Calculate explicit solution to system of first order equations
def Cinverse_negA(C,A):
    print("CINVERSENEGA")
    print("C: \n", C)
    print("A: \n", A)
    print("np.linalg.inv(C): \n", np.linalg.inv(C))
    return np.matmul(np.linalg.inv(C), -A)

def first_order_system_sol(t, t_0, y_0, C, Cinverse_negA):
    mat_exp = expm(C*(t_0-t))
    #if t<0.02:
    #    print("C: ", C)
    #    print("mat_exp: ", mat_exp)
    #print("Cinverse_negA: ", Cinverse_negA)
    return np.matmul(mat_exp, y_0-Cinverse_negA)+Cinverse_negA


### Code for the model which puts the above together ###

def model(time, time_series, N, acc_points_to_use, init_points_to_use, init_pos, coeff_lock='c_0', acc_list=None):
    print("")
    print("")
    print("!!!!!!! START !!!!!!!!!")
    num_points_to_use = [1]*N
    initial_time = 't_0'
    tolerance = 1e-6
    
    all_coeffs, fin_diff = solved_all_coeffs_analytical(time, time_series, N, acc_points_to_use, coefficient_to_lock=coeff_lock, accuracy_list=acc_list)
    print("fin_diff: \n", fin_diff)
    print("!!!!!!!!!!!!!")
    print("all_coeffs: \n", all_coeffs)

    C, A = coeff_list_to_system_matrix(all_coeffs, tolerance)
    
    try:
        print("Start of try")
        num_derivs = C.shape[0]+1
        print("num_derivs: ", num_derivs)
        fin_diff = fin_diff[:num_derivs]
        print("If the effective ODE order is different, we correspondingly remove rows from fin_diff: \n", fin_diff)
        #y_init = initial_conditions(fin_diff, num_points_to_use, initial_time)
        y_init, t0_index = initial_cond(fin_diff, init_points_to_use, init_pos, num_derivs-1)
        print("initial conditions: \n", y_init)
        #t_0 = time[int(initial_time.split("_")[1])]
        t_0 = time[t0_index]
        print("init time: ", t_0)

        Cinv_negA = Cinverse_negA(C,A)
        print("Cinv_negA: \n", Cinv_negA)
        y_sol = []

        for t in time:
            y_sol.append(first_order_system_sol(t, t_0, y_init, C, Cinv_negA))

        y_sol_forward = y_sol.copy()
        for t in (time+time[-1]):
            y_sol_forward.append(first_order_system_sol(t, t_0, y_init, C, Cinv_negA))
        print("End of try")
        
        return np.array(y_sol), np.array(y_sol_forward), (t0_index, t_0, y_init, C, Cinv_negA), all_coeffs

    # If the coefficients in front of all the derivatives are 0:
    except:
        print("In except")
        y_sol = np.ones_like(time)*(-1)*all_coeffs[0,0]    
        y_sol = np.reshape(y_sol, (-1,1,1))
        y_sol_forward = y_sol.copy()
        
        return np.array(y_sol), np.array(y_sol_forward), (None, None, None, None, None), all_coeffs


### Function that loops over and locks each coefficient, comparing which gives the best result ###

def best_c_j_to_lock(time, true_ts, N, acc_points_to_use, init_points_to_use, init_pos):
    y_sols_c_j_locked = []
    rel_errors = []
    model_params_list = []
    all_coeffs_list = []
    
    for j in range(N+1):
        y_sol, y_sol_forward, model_params, all_coeffs = model(time, true_ts, N, acc_points_to_use, init_points_to_use, init_pos, coeff_lock=f'c_{j}')
        y_sols_c_j_locked.append(y_sol)
        rel_errors.append(relative_error(y_sol[:,0,0], true_ts))
        model_params_list.append(model_params)
        all_coeffs_list.append(all_coeffs)
    best_j = rel_errors.index(np.nanmin(np.array(rel_errors)))
    print("")
    print("")
    print("Relative errors: \n", rel_errors)
    print(f"Best c_j to lock: c_{best_j}", )
    return y_sols_c_j_locked[best_j], rel_errors[best_j], model_params_list[best_j], best_j, all_coeffs_list[best_j]

### Function that loops through N (up to and including highest_N), and compares the results ###
def best_N(time, true_ts, ts_name, highest_N, plot_best_of_each_N, init_points_to_use, init_pos, save_figure=False, acc_points_to_use='few'):
    
    best_sols_dict = {}
    best_errors_dict = {}
    best_model_params_dict = {}
    best_locked_j_dict = {}
    best_all_coeffs_dict = {}
    for N in range(1,highest_N+1):
        y_best, best_rel_error, best_model_params, best_locked_j, best_all_coeffs = best_c_j_to_lock(time, true_ts, N, acc_points_to_use,  init_points_to_use, init_pos)
        best_sols_dict[N] = y_best
        best_errors_dict[N] = best_rel_error
        best_model_params_dict[N] = best_model_params
        best_locked_j_dict[N] = best_locked_j
        best_all_coeffs_dict[N] = best_all_coeffs
        
    best_sols_and_errors = best_errors_dict.items()
    
    best_N = min(best_errors_dict, key=best_errors_dict.get)
    
    best_y = best_sols_dict[best_N]
    best_err = best_errors_dict[best_N]
    best_model_params = best_model_params_dict[best_N]
    
    #print("best_sols_dict: \n", best_sols_dict)
    print("")
    print("")
    print("!!")
    print("best_locked_j_dict: ", best_locked_j_dict)
    print("best_errors_dict: \n", best_errors_dict)
    print("best_all_coeffs_dict: \n", best_all_coeffs_dict)
    print("best_N: ", best_N)
    print("best_locked_c_j: ", best_locked_j_dict[best_N])
    if plot_best_of_each_N:
        for N, best_y in best_sols_dict.items():
            plt.title(ts_name)         
            #plt.plot(time, el_price_2019_data[:len(time)], color="gray",alpha=0.5, label="Original")
            plt.plot(time, true_ts, label='True')
            plt.plot(time, best_y[:,0,0], label=f'Pred. N={N}. Locked c_{best_locked_j_dict[N]}.\nInit. cond: {init_points_to_use}, {init_pos}')
            plt.legend()
            
            if save_figure:
                if N==best_N:
                    plt.savefig(savefigs_loc+ts_name+f"__BEST__N_{N}_best_locked_{best_locked_j_dict[N]}_init_{init_points_to_use.split('%')[0]}_{init_pos}__TRAINING__rel_error_{best_errors_dict[N]}.png")
                else:
                    plt.savefig(savefigs_loc+"not_best_figures\\"+ts_name+f"__NOT_BEST__N_{N}_best_locked_{best_locked_j_dict[N]}_init_{init_points_to_use.split('%')[0]}_{init_pos}__TRAINING__rel_error_{best_errors_dict[N]}.png")       
            plt.show()
            print("Training, relative error: ", best_errors_dict[N])
            print("ODE Coefficients: \n", best_all_coeffs_dict[N])
            
    return best_y, best_err, best_model_params, best_N, best_locked_j_dict[best_N]

### Function that calculates and plots function given user-specified coefficients and initial values ###
def pre_chosen_coeffs_model(chosen_coeffs, chosen_init_conds, time_interval, t0=0):
    C, A = coeff_list_to_system_matrix(chosen_coeffs, 1e-10)
    Cinv_negA = Cinverse_negA(C,A)
    y_pred_chosen=[]
    for t in time_interval:
        y_pred_chosen.append(first_order_system_sol(t, t0, chosen_init_conds, C, Cinv_negA))
        
    plt.plot(time_interval, np.array(y_pred_chosen)[:,0,0])
    #plt.savefig(savefigs_loc+f"pre_chosen_coeffs_sol_{time_interval[-1]}.png")
    plt.show()
    return np.array(y_pred_chosen)[:,0,0]

### Function that takes in model parameters, and gives the model prediction forward in time ###
def model_params_testing(time, best_model_params, true, title, forward_in_time, best_N, best_locked_j, save_figure = False):
    sol_list = []

    t0_index = best_model_params[0]
    t0 = best_model_params[1]
    y_init = best_model_params[2]
    C = best_model_params[3]
    Cinv_negA = best_model_params[4]

    for t in time:
        sol = first_order_system_sol(t, t0, y_init, C, Cinv_negA)
        sol_list.append(sol)
        
    
    rel_error=0 
    if forward_in_time:
        middle_index = time.shape[0]//2
        rel_error = relative_error(np.array(sol_list)[middle_index:,0,0],true[middle_index:])
        print("Testing (forward in time), Relative error (on future time points): ", rel_error)
        
        #plt.plot(time, tesla_2021_2022_data[:len(time)], color="gray",alpha=0.5, label="Original")
        #plt.plot(time, true, label='Smoothed')
        plt.plot(time, np.array(sol_list)[:,0,0], label='Pred. data')
        plt.title(title)
        plt.legend()
        if save_figure:
            plt.savefig(savefigs_loc+title+f"__best_N_{best_N}_best_locked_{best_locked_j}__FIT__rel_error_{rel_error}.png")
    else:
        rel_error = relative_error(np.array(sol_list)[:,0,0],true[:])
        print("Testing (same interval), Relative error: ", rel_error)
        
        #plt.plot(time, tesla_2021_data, color="gray", alpha=0.5, label="Original")
        #plt.plot(time, true, label='Smoothed')
        plt.plot(time, np.array(sol_list)[:,0,0], label='Pred. data')
        plt.title(title)
        plt.legend()
        if save_figure:
            plt.savefig(savefigs_loc+title+f"__best_N_{best_N}_best_locked_{best_locked_j}__TESTING__rel_error_{rel_error}.png")
    
    plt.show()
    return